In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 15 12:32:11 2018
Run this file to compute the reference or potential evapotranspiration
using various methods for different years as required. ETo and PET
@author: Neha K. Nawandar
"""

#--------------------------------------------------------------------------
''' modules importing'''
import os
import math
import datetime
import pandas as pd
from openpyxl import load_workbook
#import EToComputation as eto
#import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import numpy as np
import math
import calendar
#--------------------------------------------------------------------------

In [11]:
def ReadFile():
    ''' changing path as per the file location'''
    #get the current working directory
    cwd = os.getcwd()
    cwd
    print(cwd)
    
    #change the directory and list files in current directory
    os.chdir("./")
    print(os.listdir('.'))
    
    ''' excel file'''
    #assign the excel filename to file
    file = "TempHumidityData.xlsx"
    xl = pd.ExcelFile(file)
#    print(xl.sheet_names)
    #load the file
    load_file = pd.read_excel(file)
    # Load in the workbook
    wb = load_workbook('./TempHumidityData.xlsx')
    # Get sheet names
    sheet1 = wb.get_sheet_names()
    #print(wb.get_sheet_names())
    
    ''' read the file data'''
    # Get a sheet by name and print it
    sheet = wb.get_sheet_by_name('TempHumidity')
    #print(sheet.title)
    
    # currently active sheet and other sheets
    anotherSheet = wb.active 
    #anotherSheet
    #print(anotherSheet)
    
    # Retrieve the value of a certain cell
    sheet['B3'].value
    ''' particular element'''
    # Select element 'B2' of sheet 
    c = sheet['B2']
    # row number of element
    c.row
    # column letter of element
    c.column
    # coordinates of the cell 
    c.coordinate
    # Retrieve cell value 
    sheet.cell(row=1, column=2).value
    return sheet
    ''' end of particular element'''
#--------------------------------------------------------------------------

In [12]:
#------------------------------------------------------------------------------------
#def TempHumData():
sheet_with_data = ReadFile()
#    print(sheet)
#    data of various years
for j in range(1, 8):
#    print(j)
#   initialization of temp and humidity
    t_max = []
    t_min = []
    t_mean = []
    t_dew = []
    h_relative = []
#    tmax and tmin of particular year
    for i in range(1,13):
        t_max.append((sheet_with_data.cell(row=i+2, column=2+3*(j-1)).value))
        t_min.append((sheet_with_data.cell(row=i+2, column=3+3*(j-1)).value))
        h_relative.append((sheet_with_data.cell(row=i+2, column=4+3*(j-1)).value))
#    print(len(t_max))
    for loc in range(0,12):
#        print(value)
        t_mean.append((t_max[loc]+t_min[loc])*0.5)
    for loc in range(0,12):
        t_dew.append(t_mean[loc]-(math.floor((100-h_relative[loc])/4)))

E:\PDF\OneDrive - IIT Delhi\Codes\Notebooks
['.ipynb_checkpoints', '01490100', '1_segmentation', 'details.txt', 'E0A80100', 'ET', 'ET0.ipynb', 'GUI', 'k_means', 'myfile.hdf5', 'ndvi.tif', 'NDVIcalculationLandsat8imageswithPythonandRasterio', 'ndvi_codes', 'nir.TIF', 'nir_red_codes', 'nlcd_changed.tif', 'PS_camera', 'red.TIF', 'ref', 'results', 'results_1', 'results_ndvi', 'results_tvdi', 'TempHumidityData.xlsx', 'tesseract.py', 'thermal_image_codes', 'Untitled.ipynb']


C:\Users\control\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
C:\Users\control\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).


KeyError: 'Worksheet TempHumidity does not exist.'

In [13]:
#------------------------------------------------------------------------------------
''' check functions'''
def check_day_hours(hours, arg_name):
    """
    Check that *hours* is in the range 1 to 24.
    """
    if not 0 <= hours <= 24:
        raise ValueError(
            '{0} should be in range 0-24: {1!r}'.format(arg_name, hours))

def check_doy(doy):
    """
    Check day of the year is valid.
    """
    if not 1 <= doy <= 366:
        raise ValueError(
            'Day of the year (doy) must be in range 1-366: {0!r}'.format(doy))

def check_latitude_rad(latitude):
     if not _MINLAT_RADIANS <= latitude <= _MAXLAT_RADIANS:
        raise ValueError(
            'latitude outside valid range {0!r} to {1!r} rad: {2!r}'
            .format(_MINLAT_RADIANS, _MAXLAT_RADIANS, latitude))

def check_sol_dec_rad(sd):
    """
    Solar declination can vary between -23.5 and +23.5 degrees.

    See http://mypages.iit.edu/~maslanka/SolarGeo.pdf
    """
    if not _MINSOLDEC_RADIANS <= sd <= _MAXSOLDEC_RADIANS:
        raise ValueError(
            'solar declination outside valid range {0!r} to {1!r} rad: {2!r}'
            .format(_MINSOLDEC_RADIANS, _MAXSOLDEC_RADIANS, sd))

def check_sunset_hour_angle_rad(sha):
    """
    Sunset hour angle has the range 0 to 180 degrees.

    See http://mypages.iit.edu/~maslanka/SolarGeo.pdf
    """
    if not _MINSHA_RADIANS <= sha <= _MAXSHA_RADIANS:
        raise ValueError(
            'sunset hour angle outside valid range {0!r} to {1!r} rad: {2!r}'
            .format(_MINSHA_RADIANS, _MAXSHA_RADIANS, sha))
#------------------------------------------------------------------------------------        
  

      
#------------------------------------------------------------------------------------     
''' hargreaves method specific functions'''        
def inv_rel_dist_earth_sun(day_of_year):
    """
    Calculate the inverse relative distance between earth and sun from
    day of the year.
    Based on FAO equation 23 in Allen et al (1998).
    :param day_of_year: Day of the year [1 to 366]
    :return: Inverse relative distance between earth and the sun
    :rtype: float
    """
    check_doy(day_of_year)
    return 1 + (0.033 * math.cos((2.0 * math.pi / 365.0) * day_of_year))    

def et_rad(latitude, sol_dec, sha, ird):
    """
    Estimate daily extraterrestrial radiation (*Ra*, 'top of the atmosphere
    radiation').
    Based on equation 21 in Allen et al (1998). If monthly mean radiation is
    required make sure *sol_dec*. *sha* and *irl* have been calculated using
    the day of the year that corresponds to the middle of the month.
    **Note**: From Allen et al (1998): "For the winter months in latitudes
    greater than 55 degrees (N or S), the equations have limited validity.
    Reference should be made to the Smithsonian Tables to assess possible
    deviations."
    :param latitude: Latitude [radians]
    :param sol_dec: Solar declination [radians]. Can be calculated using
        ``sol_dec()``.
    :param sha: Sunset hour angle [radians]. Can be calculated using
        ``sunset_hour_angle()``.
    :param ird: Inverse relative distance earth-sun [dimensionless]. Can be
        calculated using ``inv_rel_dist_earth_sun()``.
    :return: Daily extraterrestrial radiation [MJ m-2 day-1]
    :rtype: float
    """
    check_latitude_rad(latitude)
    check_sol_dec_rad(sol_dec)
    check_sunset_hour_angle_rad(sha)

    tmp1 = (24.0 * 60.0) / math.pi
    tmp2 = sha * math.sin(latitude) * math.sin(sol_dec)
    tmp3 = math.cos(latitude) * math.cos(sol_dec) * math.sin(sha)
    #print(tmp1 * SOLAR_CONSTANT * ird * (tmp2 + tmp3))
    return tmp1 * SOLAR_CONSTANT * ird * (tmp2 + tmp3)    
#------------------------------------------------------------------------------------           

#------------------------------------------------------------------------------------
''' temperature and angle conversion functions'''
def celsius2kelvin(celsius):
    return celsius + 273.15

def kelvin2celsius(kelvin):
    return kelvin - 273.15

def deg2rad(degrees):
    return degrees * (math.pi / 180.0)

def rad2deg(radians):
    return radians * (180.0 / math.pi)
#------------------------------------------------------------------------------------

In [14]:
#------------------------------------------------------------------------------------
''' parameters computation functions'''
def monthly_mean_daylight_hours(latitude, year=None):
    # check if entered latitude is correct and then proceed
    check_latitude_rad(latitude)
    # check for normal or leap year
    if year is None or not calendar.isleap(year):
        month_days = _MONTHDAYS
    else:
        month_days = _LEAP_MONTHDAYS
    # monthly mean daylight hours variable for all 12 months
    monthly_mean_dlh = []
    doy = 1         # Day of the year
    for mdays in month_days:
        dlh = 0.0   # Cumulative daylight hours for the month
        for daynum in range(1, mdays + 1):
            sd = sol_dec(doy)
            sha = sunset_hour_angle(latitude, sd)
            dlh += daylight_hours(sha)
            doy += 1
        # Calc mean daylight hours of the month
        monthly_mean_dlh.append(dlh / mdays)
    #print("Monthly mean daylight hours:", monthly_mean_dlh)
    return monthly_mean_dlh

def sol_dec(day_of_year):
    """
    Calculate solar declination from day of the year.
    Based on FAO equation 24 in Allen et al (1998).
    :param day_of_year: Day of year integer between 1 and 365 or 366).
    :return: solar declination [radians]
    :rtype: float
    """
    check_doy(day_of_year)
    return 0.409 * math.sin(((2.0 * math.pi / 365.0) * day_of_year - 1.39))

def sunset_hour_angle(latitude, sol_dec):
    """
    Calculate sunset hour angle (*Ws*) from latitude and solar
    declination.
    Based on FAO equation 25 in Allen et al (1998).
    :param latitude: Latitude [radians]. Note: *latitude* should be negative
        if it in the southern hemisphere, positive if in the northern
        hemisphere.
    :param sol_dec: Solar declination [radians]. Can be calculated using
        ``sol_dec()``.
    :return: Sunset hour angle [radians].
    :rtype: float
    """
    check_latitude_rad(latitude)
    check_sol_dec_rad(sol_dec)

    cos_sha = -math.tan(latitude) * math.tan(sol_dec)
    # If tmp is >= 1 there is no sunset, i.e. 24 hours of daylight
    # If tmp is <= 1 there is no sunrise, i.e. 24 hours of darkness
    # See http://www.itacanet.org/the-sun-as-a-source-of-energy/
    # part-3-calculating-solar-angles/
    # Domain of acos is -1 <= x <= 1 radians (this is not mentioned in FAO-56!)
    return math.acos(min(max(cos_sha, -1.0), 1.0))

def daylight_hours(sha):
    """
    Calculate daylight hours from sunset hour angle.

    Based on FAO equation 34 in Allen et al (1998).

    :param sha: Sunset hour angle [rad]. Can be calculated using
        ``sunset_hour_angle()``.
    :return: Daylight hours.
    :rtype: float
    """
    check_sunset_hour_angle_rad(sha)
    return (24.0 / math.pi) * sha
#------------------------------------------------------------------------------------

In [15]:
#------------------------------------------------------------------------------------
'''the constants and variables to be used'''    
_MONTHDAYS = (31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)
_LEAP_MONTHDAYS = (31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31)
# Internal constants
# Latitude
_MINLAT_RADIANS = deg2rad(-90.0)
_MAXLAT_RADIANS = deg2rad(90.0)
# Solar declination
_MINSOLDEC_RADIANS = deg2rad(-23.5)
_MAXSOLDEC_RADIANS = deg2rad(23.5)
# Sunset hour angle
_MINSHA_RADIANS = 0.0
_MAXSHA_RADIANS = deg2rad(180)
# solar constant
SOLAR_CONSTANT = 0.0820
#------------------------------------------------------------------------------------        

In [16]:
#--------------------------------------------------------------------------
#latitude
#latitude_in_deg = eval(input("Enter the latitude of your location: "))
latitude_in_deg = 21.1
latitude = deg2rad(latitude_in_deg)
#print(latitude_in_deg)

In [17]:
#compute year, month and date
#compute_date = eval(input("Enter the date: "))
#compute_month = eval(input("Enter the month: "))
#compute_year = eval(input("Enter the year: "))
num_years = 7
compute_date = 15
compute_month = 1
compute_year = 2018-num_years

In [18]:
#daylight hours
mmdlh = monthly_mean_daylight_hours(latitude,compute_year)
#print("\nMonthly mean daylight hours:", mmdlh)
#this is only for the 21.1 latitude location
mmdlh_p = [0.248,0.260,0.270,0.282,0.292,0.302,0.302,0.290,0.280,0.260,0.250,0.248]
#print("\n% mean annual daylight hours:", mmdlh_p)
#--------------------------------------------------------------------------

In [19]:
#--------------------------------------------------------------------------
''' parameters specific to hargreaves method'''
day_of_year = datetime.date(compute_year, compute_month, compute_date).timetuple().tm_yday
sol_dec = sol_dec(day_of_year)
sha = sunset_hour_angle(latitude, sol_dec)
ird = inv_rel_dist_earth_sun(day_of_year)
et_rad = et_rad(latitude, sol_dec, sha, ird)

In [20]:
''' parameters specific to priestly taylor method'''
delBydelplambda = []
solar_radiation = []
atmospheric_emissivity = []
ground_heat_flux = [0,0,0,0,0,0,0,0,0,0,0,0]
long_wave_radiation = []
net_radiation = []
for t_mean1, t_max1, t_min1, h_relative1 in zip(t_mean, t_max, t_min, h_relative):
    delBydelplambda.append((-0.00008*(t_mean1**2)) + (0.0139*t_mean1) + 0.4235)
    solar_radiation.append(0.16*et_rad*math.sqrt(t_max1-t_min1))
    atmospheric_emissivity.append((0.72+0.005*t_mean1)*(1-0.0084*h_relative1) + 0.008*h_relative1)
for t_mean1, atmospheric_emissivity1 in zip(t_mean, atmospheric_emissivity):
    long_wave_radiation.append(((4.903e-9)*(atmospheric_emissivity1-0.97)*(t_mean1+273)**4))
for long_wave_radiation1, solar_radiation1 in zip(long_wave_radiation, solar_radiation):
    net_radiation.append((0.77*solar_radiation1) - long_wave_radiation1)    
for month in range(12):
    if (month==0):
        ground_heat_flux[month] = 0.07*(t_mean[1]-t_mean[11])
    if (month==11):
        ground_heat_flux[month] = 0.07*(t_mean[0]-t_mean[month-1])
    else:
        ground_heat_flux[month] = 0.07*(t_mean[month+1]-t_mean[month-1])

NameError: name 't_mean' is not defined

In [21]:
        
''' parameters for penman-monteith method'''
#slope of the saturation vapour pressure-temperature curve kPa/C equation 1
delta = []
for t_mean1 in t_mean:
    delta.append(0.2*((0.00738*t_mean1+0.8072)**7))
#print("\n", delta)
    
##slope of the saturation vapour pressure-temperature curve kPa/C equation 2  
#delta = []
#for t_mean1 in t_mean:
#    delta.append((4098*(0.6108*math.exp((17.27*t_mean1)/(t_mean1+237.3))))/((t_mean1+237.3)**2))
#print("\n", delta)

#psychometric constant kPa/C
gamma = []
#specific heat of air at constant pressure MJ/kg/C
cp = 0.001005    
#latent heat of vapour of water MJ/kg/t_mean
Lambda = []
for t_mean1 in t_mean:
    Lambda.append(2.501-0.002364*t_mean1)
#print(Lambda)
#estimated mean atmospheric pressure kPa
elevation = 310    
p = 101.3*(((293-0.0065*elevation)/293)**5.257)
#print(p)
for Lambda1 in Lambda:
    gamma.append((cp*p)/(0.622*Lambda1))
#print(gamma)
    
#constants cn and cd for grass reference
cn = 900
cd = 0.34    
    
#saturation vapour pressure kPa equation 1
#es = []
#et_max = []
#et_min = []
#for t_max1, t_min1 in zip(t_max, t_min):
#    et_max.append(3.38639*((0.00738*t_max1+0.8072)**8 - 0.000019*(math.fabs(1.8*t_max1+48)) + 0.001316))
#    et_min.append(3.38639*((0.00738*t_min1+0.8072)**8 - 0.000019*(math.fabs(1.8*t_min1+48)) + 0.001316))
#for et_max1, et_min1 in zip(et_max, et_min):
#    es.append((et_max1+et_min1)*0.5)
##print(et_max)    
##print(et_min)
#print("\n", es)
    
#saturation vapour pressure kPa equation 2
es = []
et_max = []
et_min = []
for t_max1, t_min1 in zip(t_max, t_min):
    et_max.append(0.6108*math.exp((17.27*t_max1)/(t_max1+237.3)))
    et_min.append(0.6108*math.exp((17.27*t_min1)/(t_min1+237.3)))
for et_max1, et_min1 in zip(et_max, et_min):
    es.append((et_max1+et_min1)*0.5)
#print("\n", es)

#actual saturation vapour pressure kPa equation 1
#ea = []
#for t_dew1 in t_dew:
#    ea.append(3.38639*((0.00738*t_dew1+0.8072)**8 - 0.000019*(math.fabs(1.8*t_dew1+48)) + 0.001316))
#print("\n", ea)

#actual saturation vapour pressure kPa equation 1
ea = []
for t_dew1 in t_dew:
    ea.append(0.6108*math.exp((17.27*t_dew1)/(t_dew1+237.3)))
#print("\n", ea)

#wind speed m/s
wind_speed_u2 = [2,2,2,2,2,2,2,2,2,2,2,2]

NameError: name 't_mean' is not defined

In [66]:
'''lengths'''
#print(len(t_max))
#print(len(t_min))
#print(len(t_mean))
#print(len(t_dew))
#print(len(h_relative))
#print(len(delBydelplambda))

'''data value print'''
#print("\n",delBydelplambda)        
#print("\n",solar_radiation)        
#print("\n",atmospheric_emissivity)
#print("\n",ground_heat_flux)
#print("\n",long_wave_radiation)
#print("\n",net_radiation)
#------------------------------------------------------------------------------------

'data value print'

In [67]:
#------------------------------------------------------------------------------------
''' PET using Thornthwaite method'''    
def Thornthwaite(monthly_t, monthly_mean_dlh, year=None):
    if len(monthly_t) != 12:
        raise ValueError(
            'monthly_t should be length 12 but is length {0}.'
            .format(len(monthly_t)))
    if len(monthly_mean_dlh) != 12:
        raise ValueError(
            'monthly_mean_dlh should be length 12 but is length {0}.'
            .format(len(monthly_mean_dlh)))

    if year is None or not calendar.isleap(year):
        month_days = _MONTHDAYS
    else:
        month_days = _LEAP_MONTHDAYS

    # Negative temperatures should be set to zero
    adj_monthly_t = [t * (t >= 0) for t in monthly_t]
    # Calculate the heat index (I)
    I = 0.0
    for Tai in adj_monthly_t:
        if Tai / 5.0 > 0.0:
            I += (Tai / 5.0) ** 1.514
    a = (6.75e-07 * I ** 3) - (7.71e-05 * I ** 2) + (1.792e-02 * I) + 0.49239

#    pet = []
#    for Ta, L, N in zip(adj_monthly_t, monthly_mean_dlh, month_days):
#        # Multiply by 10 to convert cm/month --> mm/month
#        pet.append((1.6*(L/12.0)*(N/30.0)*((10.0*Ta/I)**a)*10.0))
    
    pet = [0,0,0,0,0,0,0,0,0,0,0,0]
    for loc in range(12):
        # Multiply by 10 to convert cm/month --> mm/month
        pet[loc] = 16*(monthly_mean_dlh[loc]/12)*(month_days[loc]/30)*((10.0*adj_monthly_t[loc]/I)**a)
        pet[loc] = pet[loc]/30
#    print("\nPET using Thornthwaite method:", pet)
    return pet
#------------------------------------------------------------------------------------

In [68]:
#------------------------------------------------------------------------------------
''' PET using Hargreaves method'''    
def Hargreaves(t_min, t_max, t_mean, et_rad):
    '''Note, multiplied by 0.408 to convert extraterrestrial radiation could
       be given in MJ m-2 day-1 rather than as equivalent evaporation in
       mm day-1 (alternate method when weather data is missing'''
    eto_hargreaves = []
    for t_min1, t_max1, t_mean1 in zip(t_min, t_max, t_mean):
#        eto_hargreaves1.append(0.0023*(t_mean1+17.8)*math.sqrt((t_max1 - t_min1))*0.408*et_rad)
# has a tendency to underpredict under high wind conditions (u2 > 3 m/s) and to overpredict under conditions of high relative humidity.        
        eto_hargreaves.append(0.0023*et_rad*(t_mean1+17.8)*math.sqrt(t_max1-t_min1))
#    print("\nPET using Hargreaves method:", eto_hargreaves)
    return eto_hargreaves   
#------------------------------------------------------------------------------------ 
    
#------------------------------------------------------------------------------------
''' PET using Blaney-Criddle method''' 
''' Given the limited data input to the equation, the calculated evapotranspiration
    should be regarded as only broadly accurate. Rather than a precise measure of
    evapotranspiration, the output of the equation is better thought of as providing
    an order of magnitude. The inaccuracy of the equation is exacerbated by extreme 
    variants of weather. In particular evapotranspiration is known to be exaggerated 
    by up to 40% in calm, humid, clouded areas and depreciated by 60% in windy, dry,
    sunny areas '''   
def BlaneyCriddle(mmdlh_p, t_mean):
    eto_blaneycriddle = []
    for mmdlh_p1, t_mean1 in zip(mmdlh_p, t_mean):
        eto_blaneycriddle.append(mmdlh_p1*(0.46*t_mean1 + 8.128))
#    print("\nPET using Blaney-Criddle method:", eto_blaneycriddle)
    return eto_blaneycriddle  
#------------------------------------------------------------------------------------  
 

In [69]:
   
#------------------------------------------------------------------------------------
''' PET using Linacre method'''    
def Linacre(t_dew, t_mean, mmdlh, latitude_in_deg):
    eto_linacre = []
    for t_dew1, t_mean1, mmdlh1 in zip(t_dew, t_mean, mmdlh):
        eto_linacre.append((((500*(t_mean1)+3*(mmdlh1))/(100-latitude_in_deg)) + (15*(t_mean1-t_dew1)))/(80 - t_mean1))
#    print("\nPET using Linacre method:", eto_linacre)
    return eto_linacre
#------------------------------------------------------------------------------------
   

In [70]:
 
#------------------------------------------------------------------------------------
''' PET using Hamon method'''    
def Hamon(t_mean, mmdlh):
    eto_hamon = []
    for t, daytime_length in zip(t_mean, mmdlh):
        eto_hamon.append(0.165*216.7*((6.108*math.exp((17.27*t)/(t+237.3)))/(t+237.3))*daytime_length)
#    print("\nPET using Hamon method", eto_hamon)
    return eto_hamon
#------------------------------------------------------------------------------------      

In [71]:
#------------------------------------------------------------------------------------
'''PET using Priestly-Taylor method'''    
def PriestlyTaylor(delBydelplambda,net_radiation,ground_heat_flux):
    eto_priestlytaylor = []    
    for delBydelplambda1, net_radiation1, ground_heat_flux1 in zip(delBydelplambda,net_radiation,ground_heat_flux):
        eto_priestlytaylor.append(0.408*1.26*(delBydelplambda1)*(net_radiation1-ground_heat_flux1))
#    print("\nPET using Priestly-Taylor method:", eto_priestlytaylor)
    return eto_priestlytaylor
#------------------------------------------------------------------------------------    

In [72]:
#------------------------------------------------------------------------------------
''' PET using Turc method'''
def Turc(t_mean, solar_radiation, h_relative):
    eto_turc = []
    for t_mean1, h_relative1, solar_radiation1 in zip(t_mean, h_relative, solar_radiation):
        if(h_relative1>50):
            eto_turc.append(((0.013*t_mean1)/(t_mean1+15))*(23.9*h_relative1+50))
        else:
            eto_turc.append(((0.013*t_mean1)/(t_mean1+15))*(23.9*h_relative1+50)*(1+(50-h_relative1)/70))
#    print("\nPET using Turc method:", eto_turc)
    return eto_turc
#------------------------------------------------------------------------------------

In [73]:
#------------------------------------------------------------------------------------
''' PET using Trajkovic method'''
def Trajkovic(t_mean, solar_radiation, t_max, t_min):
    eto_trajkovic = [] 
    for t_mean1, t_max1, t_min1, solar_radiation1 in zip(t_mean, t_max, t_min, solar_radiation):
        eto_trajkovic.append(0.0023*solar_radiation1*(t_mean1+17.8)*(t_max1-t_min1)**0.424)
#    print("\nPET using Trajkovic method:", eto_trajkovic)
    return eto_trajkovic
#------------------------------------------------------------------------------------

In [74]:
#------------------------------------------------------------------------------------
''' PET using Berti (modified hargreaves) method'''
def Berti(t_mean, solar_radiation, t_max, t_min):
    eto_berti = [] 
    for t_mean1, t_max1, t_min1, solar_radiation1 in zip(t_mean, t_max, t_min, solar_radiation):
        eto_berti.append(0.0023*solar_radiation1*(t_mean1+17.8)*(t_max1-t_min1)**0.517)
#    print("\nPET using Berti method:", eto_berti)
    return eto_berti
#------------------------------------------------------------------------------------    
    

In [75]:
#------------------------------------------------------------------------------------
''' PET using Schendel method'''   
def Schendel(t_mean, h_relative):
    eto_schendel = []
    for t_mean1, h_relative1 in zip(t_mean, h_relative):
        eto_schendel.append(16*t_mean1/h_relative1)
#    print("\nPET using Schendel method:", eto_schendel)
    return eto_schendel
#------------------------------------------------------------------------------------  
    

In [76]:
#------------------------------------------------------------------------------------
''' PET using Romanenko method'''   
def Romanenko(t_mean, h_relative):
    eto_romanenko = []
    for t_mean1, h_relative1 in zip(t_mean, h_relative):
        eto_romanenko.append(0.0018*(100-h_relative1)*((t_mean1+25)**2))
#    print("\nPET using Romanenko method:", eto_romanenko)
    return eto_romanenko
#------------------------------------------------------------------------------------

In [77]:
#------------------------------------------------------------------------------------
''' PET using Valiantzas 1 method'''
def Valiantzas1(t_mean, t_max, t_min, h_relative, t_dew, solar_radiation): 
    eto_valiantzas1 = []
    for t_mean1, t_max1, t_min1, h_relative1, t_dew1, solar_radiation1 in zip(t_mean, t_max, t_min, h_relative, t_dew, solar_radiation):
        eto_valiantzas1.append(0.00668*solar_radiation1*math.sqrt(((t_mean1+9.5)*(t_max1-t_min1))) - 0.0696*(t_max1-t_min1) - 0.024*(t_mean1+20)*(0.01-0.01*h_relative1) - 0.00445*solar_radiation1*math.sqrt(t_max1-t_dew1) + 0.0984*(t_mean1+17)*(1.03+0.00055*((t_max1-t_min1)**2) - (h_relative1*0.001)))
#    print("\nPET using Valiantzas 1 method:", eto_valiantzas1)
    return eto_valiantzas1
#------------------------------------------------------------------------------------

In [78]:
#------------------------------------------------------------------------------------
''' pET using Penman-Monteith method'''
def PM(delta, net_radiation, ground_heat_flux, gamma, cn, wind_speed_u2, t_mean, es, ea, cd):
    eto_pm = []
    for delta1, net_radiation1, ground_heat_flux1, gamma1, wind_speed_u21, t_mean1, es1, ea1 in zip(delta, net_radiation, ground_heat_flux, gamma, wind_speed_u2, t_mean, es, ea):
        eto_pm.append((0.408*delta1*(net_radiation1-ground_heat_flux1) + (gamma1*cn*wind_speed_u21*(es1-ea1))/(t_mean1+273))/(delta1+gamma1*(1+cd*wind_speed_u21)))
#    print("\nPET using Penman-Monteith method:", eto_pm)
    return eto_pm
#------------------------------------------------------------------------------------    
    

In [79]:
#------------------------------------------------------------------------------------
''' plot the reference evapotranspiration'''
def PlotETo(eto_hargreaves, eto_blaneycriddle, eto_linacre, eto_pt, eto_trajkovic, eto_berti,
            eto_valiantzas1, eto_pm):
#    months = ["Jan", "Feb", "March", "April", "May", "June", "July", 
#             "August", "September", "October", "November", "December"]
    params = {'legend.fontsize': 7, 'legend.handlelength': 0.5}
    rcParams.update(params)
    months = range(1,13)
    eto_plot = plot(months, eto_hargreaves, months, eto_blaneycriddle, months, eto_linacre, months, 
                    eto_pt, months, eto_trajkovic, months, eto_berti,months, 
                    eto_valiantzas1, months, eto_pm, marker='o', markersize=3)
    legend(["Hargreaves", "Blaney-Criddle", "Linacre", "Priestly-Taylor", 
                "Trajkovic", "Berti", "Valiantzas", "Penmman-Monteith"], 
            loc="upper right", labelspacing = 0.1, borderpad = 0.5)
    xlabel("Months")
    ylabel("Reference Evapotranspiration (ETo)")
#    show(eto_plot)
    savefig("D:/NEHA/PhD_DATA/JupyterNB/Results/etoPlot.jpg")
    savefig("D:/NEHA/PhD_DATA/JupyterNB/Results/etoPlot.pdf")
    savefig("D:/NEHA/PhD_DATA/JupyterNB/Results/etoPlot.png")
#------------------------------------------------------------------------------------

In [80]:
#------------------------------------------------------------------------------------
#the main function, here all eto methods are called
if __name__ == '__main__':
    ReadFile()
#    eto_thronthwaite = eto.Thornthwaite(t_mean,mmdlh) 
    eto_hargreaves = Hargreaves(t_min, t_max, t_mean, et_rad)
    eto_blaneycriddle = BlaneyCriddle(mmdlh_p,t_mean)
    eto_linacre = Linacre(t_dew, t_mean, mmdlh, latitude_in_deg)
#    eto_hamon = Hamon(t_mean, mmdlh)
    eto_pt = PriestlyTaylor(delBydelplambda,net_radiation,ground_heat_flux)
#    eto_turc = Turc(t_mean, solar_radiation, h_relative)
    eto_trajkovic = Trajkovic(t_mean, solar_radiation, t_max, t_min)
    eto_berti = Berti(t_mean, solar_radiation, t_max, t_min)
#    eto_schendel = Schendel(t_mean, h_relative)
#    eto_romanenko = Romanenko(t_mean, h_relative)
    eto_valiantzas1 = Valiantzas1(t_mean, t_max, t_min, h_relative, t_dew, solar_radiation)
    eto_pm = PM(delta, net_radiation, ground_heat_flux, gamma, cn, wind_speed_u2, t_mean, es, ea, cd)
    PlotETo(eto_hargreaves, eto_blaneycriddle, eto_linacre, eto_pt, eto_trajkovic, eto_berti, eto_valiantzas1, eto_pm)
#------------------------------------------------------------------------------------